In [1]:
# This notebook writes data to the Neo4j graph.

In [2]:
from neo4j import GraphDatabase

In [5]:
uri = "bolt://localhost:7687/"
user = "neo4j"
password = "password"

In [4]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [5]:
session = driver.session(database='brainwave')

In [10]:
# loading phone nodes into graph; csv needs to be in graph's import folder
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_nodes.csv' AS row
    WITH row
    CALL {
        WITH row
        CREATE (:Identifier {
            identifier: row.identifier,
            type: row.type
            })
    };''')

In [5]:
# create an index for phone identifiers
session.run(
    '''CREATE INDEX identifier_range_index FOR (i:Identifier) ON (i.identifier, i.type);''')

ClientError: {neo4j_code: Neo.ClientError.Schema.EquivalentSchemaRuleAlreadyExists} {message: An equivalent index already exists, 'Index( id=2, name='identifier_range_index', type='RANGE', schema=(:Identifier {identifier, type}), indexProvider='range-1.0' )'.} {gql_status: 22N70} {gql_status_description: error: data exception - equivalent index already exists. An equivalent index already exists: 'identifier_range_index'}

In [6]:
# loading phone relationships into graph
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phone_relationships_v1.csv' AS row
    WITH row
    CALL {
        WITH row
        MATCH (a:Identifier), (b:Identifier) WHERE a.identifier = row.source and b.identifier = row.target
        and a.type = row.source_type and b.type = row.target_type
        MERGE (a)-[:CONTACT {first_seen:row.first_seen, last_seen:row.last_seen, times:row.times, collection:row.collection}]->(b)
    };''')

In [7]:
# loading person nodes into graph; csv needs to be in graph's import folder
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row
    WITH row
    CALL {
        WITH row
        CREATE (:Person {
            first_name: row.first_name,
            last_name: row.last_name,
            full_name: row.full_name,
            birthday: row.birthday
            })
    };''')

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (row) { ... }', position=<SummaryInputPosition line=3, column=5, offset=88>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 88, 'line': 3, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "LOAD CSV WITH HEADERS FROM 'file:///phone_relationships_v1.csv' AS row\n    WITH row\n    CALL {\n        WITH row\n        MATCH (a:Identifier), (b:Identifier) WHERE a.identifier = row.source and b.identifier = row.target\n        and a.type = row.source_type and b.type = row.target_type\n        MERGE (a)-[:CONTACT {first_seen:row.first_s

In [8]:
session.run('''CREATE INDEX person_index FOR (n:Person) ON (n.birthday, n.full_name)''')

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (row) { ... }', position=<SummaryInputPosition line=3, column=5, offset=71>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 71, 'line': 3, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "LOAD CSV WITH HEADERS FROM 'file:///users.csv' AS row\n    WITH row\n    CALL {\n        WITH row\n        CREATE (:Person {\n            first_name: row.first_name,\n            last_name: row.last_name,\n            full_name: row.full_name,\n            birthday: row.birthday\n            })\n    };"


In [9]:
# loading user-phone relationships into graph
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///phones_users_v1.csv' AS row
    WITH row
    CALL {
        WITH row
        MATCH (p:Person), (i:Identifier) WHERE p.full_name = row.full_name AND p.birthday = row.birthday
        AND i.identifier = row.identifier AND i.type = row.type
        MERGE (p)-[:USER_OF {collection: row.collection}]->(i)
    };''')

In [10]:
# update comms relation with new comms
session.run(
    '''LOAD CSV WITH HEADERS FROM 'file:///new_comms_update.csv' AS row
    WITH row
    CALL {
    WITH row
    MATCH (a:Identifier),(b:Identifier) WHERE a.identifier = row.source AND b.identifier = row.target
    MERGE (a)-[c1:CONTACT]->(b)
    ON CREATE SET c1.last_seen = row.last_seen,c1.times = row.times,c1.first_seen=row.first_seen,c1.collection=row.collection
    ON MATCH SET c1.last_seen = row.last_seen,c1.times = row.times
};''')

Received notification from DBMS server: <GqlStatusObject gql_status='01N00', status_description='warn: feature deprecated. CALL subquery without a variable scope clause is deprecated. Use CALL (row) { ... }', position=<SummaryInputPosition line=3, column=5, offset=81>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 81, 'line': 3, 'column': 5}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: "LOAD CSV WITH HEADERS FROM 'file:///phones_users_v1.csv' AS row\n    WITH row\n    CALL {\n        WITH row\n        MATCH (p:Person), (i:Identifier) WHERE p.full_name = row.full_name AND p.birthday = row.birthday\n        AND i.identifier = row.identifier AND i.type = row.type\n        MERGE (p)-[:USER_OF {collection: row.collection}]->(i)

In [3]:
from langchain_neo4j import Neo4jGraph

In [6]:
graph = Neo4jGraph(url=uri,username=user, password=password, database='brainwave')

In [7]:
print(graph.schema)

Node properties:
Identifier {identifier: STRING, type: STRING}
Person {birthday: STRING, first_name: STRING, full_name: STRING, last_name: STRING}
Relationship properties:
CONTACT {collection: STRING, first_seen: STRING, last_seen: STRING, times: STRING}
USER_OF {collection: STRING}
The relationships:
(:Identifier)-[:CONTACT]->(:Identifier)
(:Person)-[:USER_OF]->(:Identifier)


In [8]:
from langchain_openai import ChatOpenAI

In [9]:
llm = ChatOpenAI(
    model="ai/phi4:latest",
    base_url="http://127.0.0.1:12434/engines/v1",
    api_key="ignored"
)

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
prompt = ChatPromptTemplate.from_template("""
You are an expert Neo4j developer. Use the following database schema to write
a Cypher statement to answer the user's question...

Schema: {schema}
Question: {question}""", partial_variables={'schema':graph.schema})

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
cypher_chain = prompt | llm | StrOutputParser

In [14]:
formatted_prompt = prompt.invoke({"question": "Which phones were in contact with phone 84795203656?"})
chat_response = llm.invoke(formatted_prompt)
parsed_output = StrOutputParser().invoke(chat_response.content)

In [15]:
print(parsed_output)

To find which phones were in contact with phone `84795203656`, you can use the following Cypher query. This query assumes that `84795203656` is stored as an `identifier` in a node of type `Identifier`.

```cypher
MATCH (p:Identifier {identifier: '84795203656'})-[:CONTACT]->(contact:Identifier)
RETURN contact.identifier AS ContactIdentifier
```

### Explanation:
- `MATCH (p:Identifier {identifier: '84795203656'})-[:CONTACT]->(contact:Identifier)`: This part of the query matches the `Identifier` node with the specified `identifier` and finds all nodes it has a `CONTACT` relationship with.
- `RETURN contact.identifier AS ContactIdentifier`: This returns the `identifier` property of each contact node, labeling it as `ContactIdentifier` in the result.


In [16]:
cypher_chain = prompt | llm | (lambda msg: msg.content) | StrOutputParser()


In [19]:
print(cypher_chain.invoke({"question":"Give me every phone who contacted 84795203656."}))

To find every phone that has contacted the identifier `84795203656`, you can use the following Cypher query. This query assumes that the phone numbers are stored in nodes of type `Identifier` and that the `CONTACT` relationship connects these nodes:

```cypher
MATCH (caller:Identifier)-[:CONTACT]->(target:Identifier {identifier: '84795203656'})
RETURN caller.identifier AS contacted_phone
```

This query matches all `Identifier` nodes (`caller`) that have a `CONTACT` relationship with the `Identifier` node having the identifier `84795203656` (`target`) and returns the identifiers of these calling phones.
